In [71]:
import numpy as np
import re
import matplotlib.pyplot as plt

In [72]:
def plotVertexList(list, polyList): 
    x = []
    y = []
    polyX = []
    polyY = []
    
    for index, i in enumerate(list): 
        if index % 2 == 0: 
            x.append(i)
        else: 
            y.append(i* -1) 
    for index, i in enumerate(polyList): 
        if index % 2 == 0: 
            polyX.append(i)
        else:
            polyY.append(i * -1)
    #print(x, '\n',y)
    %matplotlib widget
    plt.scatter(x,y, alpha=0.3)
    plt.plot(x,y, alpha=0.2)
    plt.plot(polyX, polyY, alpha=0.3)
    plt.show()
    
    

In [80]:
def parseBezierCurvePath(d_string):
    d_controls = re.split("([,zcCMSs-])", d_string)
    D = []

    prefix = ''
    for d in d_controls: 
        if d != '' and d != ',' and d != 'z':  
            if d == '-': 
                prefix = '-'
            else: 
                D.append(prefix + d)
                prefix = ''

    D_grouped = []

    i = 0; 
    while (i+3 <= len(D)): 
        if D[i] == 'c' or D[i] == 'C':
            D_grouped.append([D[i], D[i+1], D[i+2], D[i+3], D[i+4], D[i+5], D[i+6]])
            i += 7
        elif D[i] == 'M':
            D_grouped.append([D[i], D[i+1], D[i+2]])
            i += 3
        elif D[i] == 'S' or D[i] == 's': 
            D_grouped.append([D[i], D[i+1], D[i+2], D[i+3], D[i+4]])
            i += 5
        else: 
            i += 1
            D_grouped.append(['ERROR'])

    #print("D: ", D)
    #print("D_grouped", D_grouped)
    
    return D_grouped
    

In [87]:
#### create the vertex list 
def createVertexList(D_g, interval):
    vertex = []
    polyVert = []
    mode = "start"
    lastX = 0.0
    lastY = 0.0
    isX = True
    for d in D_g: 
        if d[0] == 'M':  # first Point
            lastX = float(d[1])
            lastY = float(d[2])
            #vertex.append(float(lastX))
            #vertex.append(float(lastY))
        if d[0] == 'C': # absolute Point
            vertex.append(lastX)
            vertex.append(lastY)
            vertex.append(float(d[1]))
            vertex.append(float(d[2]))
            vertex.append(float(d[3]))
            vertex.append(float(d[4]))
            vertex.append(float(d[5]))
            vertex.append(float(d[6]))
            lastX = float(d[5])
            lastY = float(d[6])
            #print("Absoulte Point")
        if d[0] == 'c':  # relative Points
            vertex.append(lastX)
            vertex.append(lastY)
            vertex.append(float(d[1]) + lastX)
            vertex.append(float(d[2]) + lastY)
            vertex.append(float(d[3])  + lastX)
            vertex.append(float(d[4])  + lastY)
            vertex.append(float(d[5]) + lastX)
            vertex.append(float(d[6]) + lastY)
            lastX = float(d[5]) + lastX
            lastY = float(d[6]) + lastY
        if d[0] == 'S': 
            #calc c1
            d_x = vertex[len(vertex) - 4] - vertex[len(vertex) - 2]
            d_y = vertex[len(vertex) - 3] - vertex[len(vertex) - 1]

            vertex.append(lastX)
            vertex.append(lastY)
            vertex.append(lastX - d_x) #reflection of prev. control *-1
            vertex.append(lastY - d_y)
            vertex.append(float(d[1]))
            vertex.append(float(d[2]))
            vertex.append(float(d[3]))
            vertex.append(float(d[4]))
            lastX = float(d[3])
            lastY = float(d[4])
        if d[0] == "s": 
            #calc c1
            d_x = vertex[len(vertex) - 4] - vertex[len(vertex) - 2]
            d_y = vertex[len(vertex) - 3] - vertex[len(vertex) - 1]

            vertex.append(lastX)
            vertex.append(lastY)
            vertex.append(lastX - d_x) #reflection of prev. control *-1
            vertex.append(lastY - d_y)
            vertex.append(float(d[1]) + lastX )
            vertex.append(float(d[2]) + lastY )
            vertex.append(float(d[3]) + lastX )
            vertex.append(float(d[4]) + lastY)
            lastX = vertex[len(vertex) - 2]
            lastY = vertex[len(vertex) - 1]


    #print('control vertex:', vertex)

    i = 0
    #interval = 1
    while(i < len(vertex) ): 
        points = np.array([[vertex[i], vertex[i+1], 0, 1], [vertex[i+2], vertex[i+3], 0 ,1], [vertex[i+4], vertex[i+5], 0, 1], [vertex[i+6], vertex[i+7], 0, 1]])
        m = np.array([[-1, 3, -3, 1], [3,-6,3,0],[-3, 3, 0,0],[1, 0,0,0]])
        mp = np.matmul(m, points) #points * m
        delta = points[3] - points[0]
        magDelta = np.linalg.norm(delta)
        intNorm = interval / (magDelta)

        u = 0.0
        while(u <= 1.0):
            uvec = np.array([u*u*u, u*u, u, 1.0])
            pu4 = np.matmul( uvec, mp) #mp * uvec
            polyVert.append(pu4[0])
            polyVert.append(pu4[1])
            u += intNorm
        i += 8
        
    return vertex, polyVert 

In [88]:
def getMaxVertexList(vList): 
    minX = minY = 10e10
    maxX = maxY = -10e10
    for index, i in enumerate(vList): 
        if index % 2 == 0: 
            if i < minX: 
                minX = i
            elif i > maxX: 
                maxX = i
        else: 
            if i < minY: 
                minY = i
            elif i > maxY: 
                maxY = i
    return minX, maxX, minY, maxX

In [89]:
#makes widht = 1 
def normalizeList(vList): 
    minX, maxX, minY, maxY = getMaxVertexList(vList)
    
    deltaX = maxX - minX
    deltaY = maxY - minY
    
    deltaX = maxX - minX
    listNorm = []
    for index, i in enumerate(vList): 
        if index % 2 == 0: 
            listNorm.append((i - minX) / deltaX)
        else: 
            listNorm.append((i - minY) / deltaX)
    
    return listNorm

In [90]:
def printAsArray(list, dig_round): 
    for index, i in enumerate(list):
        if index == 0: 
            s = "[" + str(round(i, dig_round)) + ", "
        else:
            s = s  + str(round(i, dig_round)) + ", "
        if index % 7 == 0 and index != 0: 
            s += "\n"
        
            
    s = s + "];"
    print(s)

In [91]:
d_base = "M78.3,21.3c-8.6-0.1-16.7,4.4-21,11.9c-4.3-7.5-12.4-12-21-11.9c-11.7,0-23.1,8-23.1,22.7c0,17.1,20.5,34.6,44.1,58.1c23.6-23.4,44.1-40.9,44.1-58.1C101.3,29.3,89.9,21.3,78.3,21.3"
#d_base = "M47.1,89.5c0,0-43.3-29-44.1-58.1s37.3-29,44.1-10.8c0,0,6.5-25,27.5-15.4C141.1,35.4,47.1,89.5,47.1,89.5z"


g_d = parseBezierCurvePath(d_base)
v, pV = createVertexList(g_d, 1.0)

pV = normalizeList(pV)
v = normalizeList(v)

print("\nNumber-Poly-Points:   ", len(pV))
plotVertexList(v, pV)


Number-Poly-Points:    524


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
printAsArray(pV, 8)

[0.73809524, 1.429e-05, 0.72600822, 0.00014064, 0.71399554, 0.00079635, 0.70207318, 0.0019737, 
0.69025708, 0.00366494, 0.67856323, 0.00586232, 0.66700757, 0.00855812, 0.65560607, 
0.01174458, 0.6443747, 0.01541397, 0.63332942, 0.01955856, 0.6224862, 0.02417059, 
0.61186099, 0.02924234, 0.60146976, 0.03476605, 0.59132847, 0.040734, 0.5814531, 
0.04713843, 0.57185959, 0.05397162, 0.56256392, 0.06122581, 0.55358205, 0.06889328, 
0.54492994, 0.07696628, 0.53662355, 0.08543707, 0.52867885, 0.0942979, 0.52111181, 
0.10354105, 0.51393838, 0.11315878, 0.50717454, 0.12314333, 0.50083623, 0.13348697, 
0.5, 0.13493492, 0.49372138, 0.12454258, 0.48701502, 0.11450826, 0.47989688, 
0.1048397, 0.47238295, 0.09554466, 0.46448916, 0.08663087, 0.4562315, 0.07810606, 
0.44762591, 0.06997798, 0.43868838, 0.06225437, 0.42943485, 0.05494297, 0.4198813, 
0.04805151, 0.41004368, 0.04158774, 0.39993796, 0.0355594, 0.38958011, 0.02997423, 
0.37898608, 0.02483997, 0.36817185, 0.02016435, 0.35715337, 0.01595512,